In [1]:
with open("../data/day16.txt") as f:
    data = f.read()

In [2]:
import numpy as np

In [3]:
arr = np.array([list(row) for row in data.split("\n")])

## Brute force

In [8]:
directions = [
    np.array((-1, 0)),
    np.array((0, -1)),
    np.array((1, 0)),
    np.array((0, 1)),
]


def is_outside_bounds(test_position, size):
    return (
        test_position[0] == -1
        or test_position[1] == -1
        or test_position[0] == size[0]
        or test_position[1] == size[1]
    )


def find_possible_directions(arr, current_position):
    possible_directions = []
    for possible_step in directions:
        possible_next_position = tuple(current_position + possible_step)
        if arr[possible_next_position] != "#":
            possible_directions.append(possible_step)
    return possible_directions


temp_arr = arr.copy()
starting_position = np.argwhere(temp_arr == "S")[0]
starting_direction = np.array([0, 1])

current_position = tuple(starting_position)
current_route = [current_position]
routes_in_consideration = [current_route]
all_routes = []

while True:
    try:
        current_route = routes_in_consideration.pop(0)
    except IndexError:
        break
    current_position = current_route[-1]
    possible_directions = find_possible_directions(temp_arr, current_position)
    for step in possible_directions:
        new_position = tuple(current_position + step)
        if new_position in current_route:
            continue  # Do not walk in loops
        new_route = current_route + [new_position]
        if temp_arr[new_position] == "E":
            all_routes.append(new_route)
        else:
            routes_in_consideration.append(new_route)
    break  # Too slow
# all_routes

In [9]:
def calculate_score(route):
    changing_directions = 0
    current_direction = (0, 1)
    for this_position, next_position in zip(route[:-1], route[1:]):
        new_direction = tuple(np.array(next_position) - np.array(this_position))
        if new_direction != current_direction:
            changing_directions += 1
            current_direction = new_direction
    return changing_directions * 1000 + len(route) - 1


# all_scores = [calculate_score(route) for route in all_routes]
# min(all_scores)

## Greedy

In [6]:
def distance(index1, index2):
    return np.sum(np.abs(np.array(index1) - np.array(index2)))


def find_route_to_try(routes_in_consideration, end_tile):
    distance_to_end_tiles = [
        distance(end_tile, route[-1]) for route in routes_in_consideration
    ]
    index = min(
        range(len(distance_to_end_tiles)), key=lambda i: distance_to_end_tiles[i]
    )
    return index

In [10]:
directions = [
    np.array((-1, 0)),
    np.array((0, -1)),
    np.array((1, 0)),
    np.array((0, 1)),
]


def is_outside_bounds(test_position, size):
    return (
        test_position[0] == -1
        or test_position[1] == -1
        or test_position[0] == size[0]
        or test_position[1] == size[1]
    )


def find_possible_directions(arr, current_position):
    possible_directions = []
    for possible_step in directions:
        possible_next_position = tuple(current_position + possible_step)
        if arr[possible_next_position] != "#":
            possible_directions.append(possible_step)
    return possible_directions


temp_arr = arr.copy()
starting_position = np.argwhere(temp_arr == "S")[0]
end_tile = np.argwhere(temp_arr == "S")[0]

current_position = tuple(starting_position)
current_route = [current_position]
routes_in_consideration = [current_route]
all_routes = []
min_score = np.inf

while True:
    try:
        index_to_try = find_route_to_try(routes_in_consideration, end_tile)
        current_route = routes_in_consideration.pop(index_to_try)
    except IndexError:
        break
    current_position = current_route[-1]
    possible_directions = find_possible_directions(temp_arr, current_position)
    for step in possible_directions:
        new_position = tuple(current_position + step)
        if new_position in current_route:
            continue  # Do not walk in loops
        new_route = current_route + [new_position]
        if calculate_score(new_route) > min_score:
            continue
        if temp_arr[new_position] == "E":
            all_routes.append(new_route)
        else:
            routes_in_consideration.append(new_route)
    break  # Too slow

# all_routes

## Pruning

In [11]:
most_efficient_states = {}


directions = [
    np.array((-1, 0)),
    np.array((0, -1)),
    np.array((1, 0)),
    np.array((0, 1)),
]


def is_outside_bounds(test_position, size):
    return (
        test_position[0] == -1
        or test_position[1] == -1
        or test_position[0] == size[0]
        or test_position[1] == size[1]
    )


def find_possible_directions(arr, current_position):
    possible_directions = []
    for possible_step in directions:
        possible_next_position = tuple(current_position + possible_step)
        if arr[possible_next_position] != "#":
            possible_directions.append(possible_step)
    return possible_directions


temp_arr = arr.copy()
starting_position = np.argwhere(temp_arr == "S")[0]
end_tile = np.argwhere(temp_arr == "S")[0]

current_position = tuple(starting_position)
current_route = [current_position]
routes_in_consideration = [current_route]
all_routes = []
all_scores = []
while True:
    try:
        current_route = routes_in_consideration.pop(0)
    except IndexError:
        break
    current_position = current_route[-1]
    possible_directions = find_possible_directions(temp_arr, current_position)
    for step in possible_directions:
        new_position = tuple(current_position + step)
        if new_position in current_route:
            continue  # Do not walk in loops
        new_route = current_route + [new_position]
        current_state = (new_position, tuple(step))
        current_score = calculate_score(new_route)

        if current_score >= most_efficient_states.get(current_state, np.inf):
            # Do not save inefficient routes
            continue
        if temp_arr[new_position] == "E":
            all_routes.append(new_route)
            all_scores.append(calculate_score(new_route))
            print(current_score)
        else:
            most_efficient_states[current_state] = current_score
            routes_in_consideration.append(new_route)

# all_routes

111404
109404
107408
105420
103444
101452
99452
97464
95476


In [13]:
most_efficient_score = min(all_scores)
most_efficient_score

95476

# Part 2

In [ ]:
all_minimal_score_routes = []
current_position = tuple(starting_position)
current_route = [current_position]
routes_in_consideration = [current_route]

while True:
    try:
        current_route = routes_in_consideration.pop(0)
    except IndexError:
        break
    current_position = current_route[-1]
    possible_directions = find_possible_directions(temp_arr, current_position)
    for step in possible_directions:
        new_position = tuple(current_position + step)
        if new_position in current_route:
            continue  # Do not walk in loops
        new_route = current_route + [new_position]
        current_state = (new_position, tuple(step))
        current_score = calculate_score(new_route)

        # This is the only crucial change: >= turns into >
        # We need ALL paths that go throught the most efficient paths
        # This is still of acceptable time because we ran the code for only one efficient path beforehand
        # This might be of acceptable performance if ran in combination with a greedy approach
        if (
            current_score > most_efficient_states.get(current_state, np.inf)
            or current_score > most_efficient_score
        ):
            # Do not save inefficient routes
            continue
        if temp_arr[new_position] == "E":
            all_minimal_score_routes.append(new_route)
            # all_scores.append(calculate_score(new_route))
            print(current_score)
        else:
            most_efficient_states[current_state] = current_score
            routes_in_consideration.append(new_route)

95476
95476
95476
95476


In [15]:
unique_positions = set(
    tuple_ for sublist in all_minimal_score_routes for tuple_ in sublist
)
len(unique_positions)

511